In [17]:
from obspy import read
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import numpy as np

# Leer traza
tr = read("../data/raw/LP/2018-03-03-2214-05S.LC05__001_00_01")[0]

# Preprocesamiento
tr.detrend("demean")
tr.detrend("linear")
tr.taper(max_percentage=0.05, type='cosine')  # usamos 'cosine' para evitar conflicto con scipy
tr.filter("bandpass", freqmin=1.0, freqmax=50.0)

# Parámetros
data = tr.data
fs = tr.stats.sampling_rate
times = np.linspace(0, tr.stats.npts / fs, tr.stats.npts)

# Espectrograma
f, t, Sxx = spectrogram(data, fs=fs, window='hann', nperseg=256, noverlap=128)

# Figura con dos subplots
fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharex=False, gridspec_kw={'height_ratios': [1, 1.5]})

# Señal preprocesada
axs[0].plot(times, data, color='mediumorchid')
axs[0].set_title("Preprocessed Signal")
axs[0].set_ylabel("Amplitude")
axs[0].grid(True)

# Espectrograma
pcm = axs[1].pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='magma')
axs[1].set_title("Spectrogram")
axs[1].set_ylabel("Frequency [Hz]")
axs[1].set_xlabel("Time [s]")

# Colorbar
cbar = fig.colorbar(pcm, ax=axs[1], label="Power/Frequency (dB/Hz)")
plt.tight_layout()
plt.show()




<Figure size 640x480 with 0 Axes>

In [6]:
import numpy as np
from scipy.stats import kurtosis, skew
from obspy import read

st = read("../data/raw""/TC/2018-03-04-0956-26S.LC05__001_00_01")
tr = st[0]
data = tr.data.astype(float)

# Tiempo dominio
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
max_ = np.max(data)
min_ = np.min(data)
range_ = max_ - min_
energy = np.sum(data ** 2)
rms = np.sqrt(np.mean(data ** 2))
kurt = kurtosis(data)
sk = skew(data)

# Frecuencia dominio
fft = np.fft.fft(data)
freqs = np.fft.fftfreq(len(data), d=1/tr.stats.sampling_rate)
fft_magnitude = np.abs(fft)

# Sólo frecuencias positivas
pos_mask = freqs > 0
freqs = freqs[pos_mask]
fft_magnitude = fft_magnitude[pos_mask]

dominant_freq = freqs[np.argmax(fft_magnitude)]
spectral_centroid = np.sum(freqs * fft_magnitude) / np.sum(fft_magnitude)

features = {
    "mean": mean,
    "median": median,
    "std": std,
    "max": max_,
    "min": min_,
    "range": range_,
    "energy": energy,
    "rms": rms,
    "kurtosis": kurt,
    "skewness": sk,
    "dominant_freq": dominant_freq,
    "spectral_centroid": spectral_centroid
}

print(features)


{'mean': -138.47171875, 'median': -138.0, 'std': 3.2813126444912926, 'max': -117.0, 'min': -156.0, 'range': 39.0, 'energy': 245570354.0, 'rms': 138.51059131434678, 'kurtosis': 3.4199844094613656, 'skewness': 0.010364870835406069, 'dominant_freq': 3.609375, 'spectral_centroid': 29.410342473771976}


In [7]:
import os
import pandas as pd
import numpy as np
from obspy import read
from scipy.stats import kurtosis, skew

# Ruta base padre donde están las carpetas HY, LP, TC, TR, VT
base_path = "../data/raw"
folders = ['HY', 'LP', 'TC', 'TR', 'VT']

# Lista para guardar diccionarios con features
features_list = []

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        print(f"La carpeta {folder} no existe en {base_path}")
        continue

    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    if not files:
        print(f"No se encontraron archivos en la carpeta {folder}.")
        continue

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        print(f"Procesando archivo: {file_path}")

        try:
            st = read(file_path)
            tr = st[0]
            data = tr.data.astype(float)
            sr = tr.stats.sampling_rate

            # Calcular features tiempo dominio
            mean = np.mean(data)
            median = np.median(data)
            std = np.std(data)
            max_ = np.max(data)
            min_ = np.min(data)
            range_ = max_ - min_
            energy = np.sum(data ** 2)
            rms = np.sqrt(np.mean(data ** 2))
            kurt = kurtosis(data)
            sk = skew(data)

            # Calcular features frecuencia dominio
            fft = np.fft.fft(data)
            freqs = np.fft.fftfreq(len(data), d=1/sr)
            fft_magnitude = np.abs(fft)

            # Filtrar frecuencias positivas
            pos_mask = freqs > 0
            freqs = freqs[pos_mask]
            fft_magnitude = fft_magnitude[pos_mask]

            if len(fft_magnitude) > 0 and np.sum(fft_magnitude) != 0:
                dominant_freq = freqs[np.argmax(fft_magnitude)]
                spectral_centroid = np.sum(freqs * fft_magnitude) / np.sum(fft_magnitude)
            else:
                dominant_freq = 0
                spectral_centroid = 0

            # Guardar todo en un diccionario
            feature_dict = {
                "etiqueta": folder,
                "id_evento": file_name,
                "mean": mean,
                "median": median,
                "std": std,
                "max": max_,
                "min": min_,
                "range": range_,
                "energy": energy,
                "rms": rms,
                "kurtosis": kurt,
                "skewness": sk,
                "dominant_freq": dominant_freq,
                "spectral_centroid": spectral_centroid
            }

            features_list.append(feature_dict)

        except Exception as e:
            print(f"Error procesando {file_path}: {e}")

# Crear dataframe con todas las features
df_features = pd.DataFrame(features_list)

# Guardar a CSV si quieres
df_features.to_csv("../data/processed/features_sismicas.csv", index=False)

print("Extracción finalizada. Dataset creado con forma:", df_features.shape)
print(df_features.head())


Procesando archivo: ../data/raw\HY\2018-03-04-0532-30S.LC05__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-04-2118-44S.LC05__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-06-0544-55S.LC05__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-08-2155-43S.LC09__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-09-0614-52S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-09-1944-45S.LC05__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-09-2134-23S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-10-0105-50S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-10-1907-49S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-10-2146-56S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-11-0115-53S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-11-0549-15S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-11-1229-02S.LC10__001_00_01
Procesando archivo: ../data/raw\HY\2018-03-12-0654-15S.LC10__001_00_01
Proces

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Supongamos que df_features ya está cargado o creado
# Cargar si es necesario
df_features = pd.read_csv("../data/processed/features_sismicas.csv")

# Separar features (X) y etiqueta (y)
X = df_features.drop(columns=["etiqueta", "id_evento"])
y = df_features["etiqueta"]

# División estratificada para balancear clases en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Distribución de clases en entrenamiento:")
print(y_train.value_counts())
print("\nDistribución de clases en prueba:")
print(y_test.value_counts())

# Entrenar Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predecir
y_pred = clf.predict(X_test)

# Reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión (opcional)
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
cm_df = pd.DataFrame(cm, index=clf.classes_, columns=clf.classes_)
print("\nMatriz de confusión:")
print(cm_df)


Distribución de clases en entrenamiento:
etiqueta
VT    2149
TC    1758
LP     462
TR     377
HY     170
Name: count, dtype: int64

Distribución de clases en prueba:
etiqueta
VT    537
TC    440
LP    115
TR     94
HY     43
Name: count, dtype: int64

Reporte de clasificación:
              precision    recall  f1-score   support

          HY       0.40      0.05      0.08        43
          LP       0.45      0.26      0.33       115
          TC       0.59      0.70      0.64       440
          TR       0.44      0.07      0.13        94
          VT       0.76      0.89      0.82       537

    accuracy                           0.67      1229
   macro avg       0.53      0.39      0.40      1229
weighted avg       0.64      0.67      0.63      1229


Matriz de confusión:
    HY  LP   TC  TR   VT
HY   2   2   26   1   12
LP   0  30   79   0    6
TC   3  25  306   6  100
TR   0   0   56   7   31
VT   0   9   48   2  478


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd

# Suponemos que df_features ya está cargado
X = df_features.drop(columns=["etiqueta", "id_evento"])
y = df_features["etiqueta"]

# División estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalado (muy importante para SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [6]:
from sklearn.svm import SVC

svm_clf = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
svm_clf.fit(X_train_scaled, y_train)

y_pred_svm = svm_clf.predict(X_test_scaled)
print("Reporte clasificación SVM:")
print(classification_report(y_test, y_pred_svm))


Reporte clasificación SVM:
              precision    recall  f1-score   support

          HY       0.00      0.00      0.00        43
          LP       1.00      0.03      0.05       115
          TC       0.52      0.63      0.57       440
          TR       0.00      0.00      0.00        94
          VT       0.64      0.83      0.72       537

    accuracy                           0.59      1229
   macro avg       0.43      0.30      0.27      1229
weighted avg       0.56      0.59      0.53      1229



c:\Users\nicol\anaconda3\envs\volcan-seismic\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
c:\Users\nicol\anaconda3\envs\volcan-seismic\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
c:\Users\nicol\anaconda3\envs\volcan-seismic\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Ahora dividir
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Entrenar XGBoost con etiquetas numéricas
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)

# Para mostrar resultados con etiquetas originales
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=le.classes_))


import xgboost as xgb

# Para XGBoost no siempre es necesario escalar
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)
print("Reporte clasificación XGBoost:")
print(classification_report(y_test, y_pred_xgb))


c:\Users\nicol\anaconda3\envs\volcan-seismic\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:32:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



              precision    recall  f1-score   support

          HY       0.50      0.12      0.19        43
          LP       0.44      0.34      0.38       115
          TC       0.62      0.72      0.66       440
          TR       0.32      0.13      0.18        94
          VT       0.80      0.86      0.83       537

    accuracy                           0.68      1229
   macro avg       0.54      0.43      0.45      1229
weighted avg       0.65      0.68      0.66      1229



c:\Users\nicol\anaconda3\envs\volcan-seismic\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:32:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



Reporte clasificación XGBoost:
              precision    recall  f1-score   support

           0       0.50      0.12      0.19        43
           1       0.44      0.34      0.38       115
           2       0.62      0.72      0.66       440
           3       0.32      0.13      0.18        94
           4       0.80      0.86      0.83       537

    accuracy                           0.68      1229
   macro avg       0.54      0.43      0.45      1229
weighted avg       0.65      0.68      0.66      1229

